# Brave Search MCP Server and Client Demo

This notebook demonstrates how to use the Brave Search MCP server and clients in Google Colab.

## 1. Install Dependencies

First, let's install the required dependencies:

In [1]:
!pip install langchain langchain_openai autogen requests flask python-dotenv ipywidgets


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


## 2. Set Up Environment Variables

Create a `.env` file with your API keys:

In [2]:
%%writefile .env
# API Keys for MCP Server and Client
BRAVE_SEARCH_API_KEY=your_brave_search_api_key_here
OPENAI_API_KEY=your_openai_api_key_here

Overwriting .env


## 3. Start the MCP Server

Now let's start the MCP server:

In [3]:
import os
import threading
import time
import requests
from flask import Flask, request, jsonify
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
BRAVE_SEARCH_API_KEY = os.getenv("BRAVE_SEARCH_API_KEY")
if not BRAVE_SEARCH_API_KEY or BRAVE_SEARCH_API_KEY == "your_brave_search_api_key_here":
    print("⚠️ Warning: BRAVE_SEARCH_API_KEY not properly set in .env file")

BRAVE_SEARCH_ENDPOINT = "https://api.search.brave.com/res/v1/web/search"

# Create a Flask server
app = Flask(__name__)

# Add a basic route for testing
@app.route('/', methods=['GET'])
def hello():
    return jsonify({"status": "MCP server is running"}), 200

@app.route('/search', methods=['POST'])
def search():
    data = request.json
    query = data.get('query', '')
    count = data.get('count', 5)

    # Call the actual Brave Search API
    headers = {
        "Accept": "application/json",
        "X-Subscription-Token": BRAVE_SEARCH_API_KEY
    }
    params = {
        "q": query,
        "count": count
    }

    response = requests.get(BRAVE_SEARCH_ENDPOINT, headers=headers, params=params)

    if response.status_code == 200:
        # Process the response to match our expected format
        results = response.json()
        processed_results = {
            "results": [
                {
                    "title": item.get("title", ""),
                    "url": item.get("url", ""),
                    "description": item.get("description", "")
                }
                for item in results.get("web", {}).get("results", [])
            ]
        }
        return jsonify(processed_results)
    else:
        return jsonify({"error": f"Error: {response.status_code} - {response.text}"}), 500

# Function to run the Flask server in a separate thread
def run_flask_server():
    app.run(host='127.0.0.1', port=8080, debug=False, use_reloader=False)

# Start the server in a background thread
server_thread = threading.Thread(target=run_flask_server)
server_thread.daemon = True
server_thread.start()

# Give the server time to start
print("Starting MCP server...")
time.sleep(5)  # Increased wait time
print("MCP Server is now running at http://127.0.0.1:8080")

Starting MCP server...
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.


MCP Server is now running at http://127.0.0.1:8080


## 4. Test the Server

Let's test the server to make sure it's working correctly:

In [4]:
# Test direct API calls to confirm server is working
print("Testing direct API calls to the MCP server...")

# First, test the root endpoint
try:
    root_response = requests.get("http://127.0.0.1:8080/")
    print(f"Root endpoint test - Status: {root_response.status_code}")
    if root_response.status_code == 200:
        print("Root endpoint is working!")
    else:
        print(f"Root endpoint error: {root_response.text}")
except Exception as e:
    print(f"Connection error to root endpoint: {str(e)}")

# Then, test the search endpoint
try:
    search_response = requests.post(
        "http://127.0.0.1:8080/search",
        headers={"Content-Type": "application/json"},
        json={"query": "test query", "count": 1}
    )
    print(f"Search endpoint test - Status: {search_response.status_code}")
    if search_response.status_code == 200:
        print("Search endpoint is working correctly!")
        print(f"Response preview: {str(search_response.json())[:200]}...")
    else:
        print(f"Search endpoint error: {search_response.text}")
except Exception as e:
    print(f"Connection error to search endpoint: {str(e)}")

Testing direct API calls to the MCP server...
Root endpoint test - Status: 200
Root endpoint is working!
Search endpoint test - Status: 200
Search endpoint is working correctly!
Response preview: {'results': [{'description': 'Hello Experts, I have one scenario/requirement to create Destinations for CTMS configuration. The Destinations TransportManagementService, ContentAssemblyService and Clou...


## 5. LangChain Client Implementation

Now let's implement the LangChain client:

In [5]:
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent

def brave_search(query):
    """Use Brave Search to find information on the web via local MCP server."""
    url = "http://127.0.0.1:8080/search"
    headers = {"Content-Type": "application/json"}
    payload = {"query": query, "count": 5}

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        results = response.json()
        formatted_results = []
        for i, result in enumerate(results.get("results", [])):
            formatted_results.append(f"{i+1}. {result.get('title')}: {result.get('url')}\n{result.get('description')}\n")
        return "\n".join(formatted_results)
    else:
        return f"Error: {response.status_code} - {response.text}"

def run_langchain_demo(query):
    # Create a LangChain tool
    brave_tool = Tool(
        name="BraveSearch",
        description="Useful for searching the web about current events, data, or any information you need to answer questions.",
        func=brave_search
    )

    # Initialize the LLM with API key from .env
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key or openai_api_key == "your_openai_api_key_here":
        print("⚠️ Warning: OPENAI_API_KEY not properly set in .env file")

    llm = ChatOpenAI(api_key=openai_api_key, temperature=0)

    # Initialize the agent with the Brave Search tool
    agent = initialize_agent(
        [brave_tool],
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    return agent.run(query)

## 6. AutoGen Client Implementation

Now let's implement the AutoGen client:

In [6]:
import autogen

# Define the function to query the local MCP server
def query_brave_search(query):
    """Query the local Brave Search MCP server."""
    url = "http://127.0.0.1:8080/search"
    headers = {"Content-Type": "application/json"}
    payload = {"query": query, "count": 5}

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

def run_autogen_demo(query):
    # Configure the LLM using API key from .env
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key or openai_api_key == "your_openai_api_key_here":
        print("⚠️ Warning: OPENAI_API_KEY not properly set in .env file")

    config_list = [
        {
            "model": "gpt-4",
            "api_key": openai_api_key
        }
    ]

    # Create AutoGen agents
    assistant = autogen.AssistantAgent(
        name="assistant",
        llm_config={"config_list": config_list}
    )

    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=10,
        code_execution_config={"work_dir": "coding"},
        function_map={"brave_search": query_brave_search}
    )

    # Start the conversation
    user_proxy.initiate_chat(
        assistant,
        message=f"Use the brave_search function to find information about: {query}, then summarize the findings."
    )

## 7. Demo UI

Finally, let's create a demo UI:

In [7]:
from IPython.display import HTML, display
import ipywidgets as widgets

# Create demo UI
demo_type = widgets.RadioButtons(
    options=['LangChain', 'AutoGen'],
    description='Demo Type:',
    disabled=False
)

query_input = widgets.Text(
    value='What are the latest developments in quantum computing?',
    placeholder='Enter your search query',
    description='Query:',
    disabled=False
)

run_button = widgets.Button(
    description='Run Demo',
    disabled=False,
    button_style='success',
    tooltip='Click to run the selected demo with your query'
)

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print(f"Running {demo_type.value} demo with query: {query_input.value}")
        if demo_type.value == 'LangChain':
            result = run_langchain_demo(query_input.value)
            print(f"\nResult: {result}")
        else:
            run_autogen_demo(query_input.value)

run_button.on_click(on_button_click)

# Display the UI
display(demo_type, query_input, run_button, output)

RadioButtons(description='Demo Type:', options=('LangChain', 'AutoGen'), value='LangChain')

Text(value='What are the latest developments in quantum computing?', description='Query:', placeholder='Enter …

Button(button_style='success', description='Run Demo', style=ButtonStyle(), tooltip='Click to run the selected…

Output()